In [1]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.4.1'

In [40]:
path = 'HandGestureRecognitionNeuralNetwork/dataset/'
#path = path + 'left/'

In [41]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(shear_range=0.2, zoom_range=0.4, height_shift_range=20 , validation_split = 0.15, horizontal_flip = True)

train_generator = datagen.flow_from_directory(path, target_size=(256, 256),
    class_mode = 'categorical', shuffle=True, batch_size = 4, subset = 'training')


val_generator = datagen.flow_from_directory(path, class_mode = 'categorical', subset = 'validation', shuffle = True)

Found 134 images belonging to 4 classes.
Found 22 images belonging to 4 classes.


In [45]:
k = next(train_generator)
INPUT_SHAPE = k[0].shape
label_shape = k[1].shape

In [46]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_generator, output_signature = (
                tf.TensorSpec(shape = INPUT_SHAPE, dtype = tf.float32),
                tf.TensorSpec(shape = label_shape, dtype = tf.int32)))

In [47]:
train_dataset.element_spec

(TensorSpec(shape=(4, 256, 256, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(4, 4), dtype=tf.int32, name=None))

In [48]:
for k in train_dataset.take(1):
    print(k[1].shape)


(4, 4)


In [49]:
from tensorflow.keras import Model, Input, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback

In [50]:
act = tf.nn.leaky_relu
model = Sequential([
    Conv2D(64, (3,3), activation = act, input_shape = (256, 256, 3)),
    MaxPooling2D((2,2)),
    Conv2D(32, (3,3), activation = act),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(16, activation = act),
    Dense(4, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics =['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 125, 125, 32)      18464     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 62, 62, 32)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 123008)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                1968144   
_________________________________________________________________
dense_11 (Dense)             (None, 4)                

In [54]:
early_stopper = EarlyStopping(patience = 2, monitor = 'loss', min_delta = 0.01)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', )
callbacks = [reduce_lr]
history = model.fit(train_generator,validation_data = val_generator, batch_size = 8,  epochs = 20, callbacks = callbacks, verbose = 1, shuffle = True)

Epoch 1/20
34/34 [==============================] - 9s 268ms/step - loss: 4.1582 - accuracy: 0.7985 - val_loss: 2.6838 - val_accuracy: 0.8636
Epoch 2/20
34/34 [==============================] - 8s 224ms/step - loss: 1.7702 - accuracy: 0.8582 - val_loss: 2.4735 - val_accuracy: 0.8636
Epoch 3/20
34/34 [==============================] - 8s 231ms/step - loss: 1.8531 - accuracy: 0.8731 - val_loss: 4.0332 - val_accuracy: 0.7727
Epoch 4/20
34/34 [==============================] - 8s 235ms/step - loss: 2.8502 - accuracy: 0.8955 - val_loss: 3.9201 - val_accuracy: 0.7727
Epoch 5/20
34/34 [==============================] - 8s 227ms/step - loss: 2.6164 - accuracy: 0.8209 - val_loss: 5.4931 - val_accuracy: 0.7727
Epoch 6/20
34/34 [==============================] - 8s 225ms/step - loss: 2.3640 - accuracy: 0.8433 - val_loss: 1.6021 - val_accuracy: 0.9091
Epoch 7/20
34/34 [==============================] - 8s 223ms/step - loss: 2.5725 - accuracy: 0.8433 - val_loss: 3.3438 - val_accuracy: 0.8636
Epoch 

In [ ]:
### tensorflow lite interpreter za raspberry
### nej shrani augmentatione da mamo za trenirat

## TensorFlow Lite Converter

In [ ]:
import pathlib
export_dir = 'models/saved_model/'
lite_filepath = 'models/foo.tflite'

def convert(model, export_dir = export_dir, lite_filepath = lite_filepath):
    tf.saved_model.save(model, export_dir)
    lite_filepath = 'models/foo.tflite'
    tflite_model_file = pathlib.Path(lite_filepath)
    tflite_model_file.write_bytes(tflite_model)

In [59]:
!mkdir -p models
!chmod +x models

In [61]:
convert(model)

INFO:tensorflow:Assets written to: models/saved_model/assets


In [ ]:
## check quantization

In [ ]:
## using the interpreter

## to daš v raspberry

interpreter = tf.lite.Interpreter(lite_filepath)
signature = interpreter.get_signature_runner()
output = signature(x = image)
print(output['result'])

In [ ]:
https://www.tensorflow.org/lite/guide/inference#load_and_run_a_model_in_python

In [ ]:
image = Image.open(data_folder + "test.jpg").convert('RGB').resize((width, height))
